In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, time, datetime
import matplotlib
from helpers import return_good_regions

In [ ]:
# Load with various verbose summaries
def load_csv(file_path, columns, na_vals=['NA',], parse_d=True, verbose=0):
    dtype_map = {}
    for col in columns:
        dtype_map[col] = np.float64
    df = pd.read_csv(file_path,
                    dtype=dtype_map,
                    parse_dates=parse_d,
                    na_values=na_vals)
    if verbose >= 1:
        #print("\nHead(10)")
        #print(df.head(10))
        print("\nDescribe")
        print(df.describe().round(2))
        print("\ndf.isna().sum(axis=0)")
        print(df.isna().sum(axis=0))
    if verbose >= 2:
        plt.imshow(~df.isna(), aspect='auto')
        plt.xlabel("variables")
        plt.ylabel("cases")
        plt.gray()
        plt.show()
    if 'date_time' in df.columns:
        df['date_time'] = pd.to_datetime(df['date_time'])
    return df

In [ ]:
base = '/Users/truggles/Downloads/for_tyler_Oct07_overimputes/'

regions=['CISO']
df = load_csv(base+'csv_MASTER_v12_2day.csv', regions, ['NA',], True, 0)
df['date_time'] = pd.to_datetime(df['date_time'])

In [ ]:
#def intro_plot(winter, spring, summer, fall, annual)


def master_plot(df, col, cols, all_regions):
    
    plt.close()
    matplotlib.rcParams['figure.figsize'] = (15.0, 7.0)
    
    #fig, ax = plt.subplots(figsize=(15,7))
    ax = plt.subplot(212)
    ax.margins(0.05)
    annual_plot(ax, df, col)
    ax1 = plt.subplot(241)
    seasonal_plot(ax1, df, cols, all_regions, 1, 2016)
    ax1.set_ylim(ax1.get_ylim()[0], 2.5)
    ax1.set_ylabel('Normalized demand')
    ax2 = plt.subplot(242, sharey=ax1)
    plt.setp(ax2.get_yticklabels(), visible=False)
    seasonal_plot(ax2, df, cols, all_regions, 4, 2016)
    plt.legend(loc='upper left', ncol=2, prop={'size': 14})
    ax3 = plt.subplot(243, sharey=ax1)
    plt.setp(ax3.get_yticklabels(), visible=False)
    seasonal_plot(ax3, df, cols, all_regions, 7, 2016)
    ax4 = plt.subplot(244, sharey=ax1)
    plt.setp(ax4.get_yticklabels(), visible=False)
    seasonal_plot(ax4, df, cols, all_regions, 10, 2016)
    plt.savefig('intro_plot.pdf')

def annual_plot(ax, df, col):
    peaks = []
    troughs = []
    mean = []
    days = []
    
    prev_day_of_year = -1
    prev_dt = ''
    dem_vals = []
    cnt = 0
    #print("DF num NAN {}".format(df.isna().sum()))
    for idx in df.index:
        
        d = df.loc[idx, 'date_time']
        day_of_year = d.toordinal() - date(d.year, 1, 1).toordinal() + 1
        
        if idx == 0:
            prev_day_of_year = day_of_year
            prev_dt = d
        
        # Fill values if we iterated
        if day_of_year != prev_day_of_year:
            days.append(prev_dt)
            peaks.append( max(dem_vals) )
            troughs.append( min(dem_vals) )
            mean.append( np.nanmean(dem_vals) )
            dem_vals.clear()
            prev_day_of_year = day_of_year
            prev_dt = d
        
        dem_vals.append( df.loc[idx, col] )
        cnt += 1
        if cnt > 17520:
            break
    
    ax.set_ylabel('Demand (MW)')
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    #plt.title(col)
    ax.plot(days, mean, 'k-', label='Mean Demand', linewidth=1.0)    
    ax.fill_between(days, troughs, peaks, facecolor='orange', alpha=0.6, label='Daily Range')
    plt.tight_layout()

def seasonal_plot(ax, df, cols, all_regions, month, year):

    ax.yaxis.grid(True)
    ax.set_xlabel('Hour of day (UTC)')
    
    for col in cols:
        if hasattr(all_regions, col):
            all_regions.remove(col)
    
    dem_vals = {}
    other_dem_vals = {}
    for col in cols:
        dem_vals[col] = [[] for _ in range(24)]
    for col in all_regions:
        other_dem_vals[col] = [[] for _ in range(24)]
    for idx in df.index:
        
        d = df.loc[idx, 'date_time']
        if not (d.month == month and d.year == year):
            continue
        
        for col in cols:
            dem_vals[col][d.hour].append(df.loc[idx, col])
        for col in all_regions:
            other_dem_vals[col][d.hour].append(df.loc[idx, col])
    
    
    
    
    #ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    mmap = {
        1:'January',
        4:'April',
        7:'July',
        10:'October'
    }
    plt.title(mmap[month])
    
    # Plot those in gray first
    for col in all_regions:
        dem_vals_mean = []
        for i in range(24):
            dem_vals_mean.append( np.nanmean(other_dem_vals[col][i]))
            
        # Normalize by mean value
        mean = np.nanmean(df[col])
        ax.plot([i for i in range(24)], dem_vals_mean/mean, label='_nolegend_', linewidth=1.0, color='gray')
    
    for col in cols:
        dem_vals_mean = []
        for i in range(24):
            dem_vals_mean.append( np.nanmean(dem_vals[col][i]))
            
        # Normalize by mean value
        mean = np.nanmean(df[col])
        ax.plot([i for i in range(24)], dem_vals_mean/mean, label=col, linewidth=2.0)
    plt.tight_layout()
    
    
regions = return_good_regions()
regions_for_seasons = ['PJM','MISO','ERCO','SWPP','SOCO','CISO','NYIS','TVA','FPL','ISNE',]#'DUK']
region_for_annual = 'TVA'

#regions = regions_for_seasons
#regions_for_seasons = ['PJM', 'MISO']
matplotlib.rcParams.update({'font.size': 15})
master_plot(df, region_for_annual, regions_for_seasons, regions)

